## Importando Dados e Bibliotecas

In [1]:
import pandas as pd
import requests, json
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import *
import urllib.request, urllib.parse, urllib.error

request = requests.get("https://brasil.io/api/dataset/covid19/caso_full/data?city=Assis")

In [2]:
data_json = json.loads(request.content)
data = pd.DataFrame.from_dict(data_json['results'], orient='columns')
data

,city,city_ibge_code,date,epidemiological_week,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,new_confirmed,new_deaths,order_for_place,place_type,state
0,Assis,3504008,2020-07-01,27,104386,False,True,182,174.35288,2020-06-30,0.0604,11,0,0,91,city,SP
1,Assis,3504008,2020-06-30,27,104386,True,False,182,174.35288,2020-06-30,0.0604,11,9,3,90,city,SP
2,Assis,3504008,2020-06-29,27,104386,False,False,173,165.73104,2020-06-29,0.0462,8,2,0,89,city,SP
3,Assis,3504008,2020-06-28,27,104386,False,False,171,163.81507,2020-06-28,0.0468,8,2,0,88,city,SP
4,Assis,3504008,2020-06-27,26,104386,False,False,169,161.89911,2020-06-27,0.0473,8,2,0,87,city,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Assis,3504008,2020-04-06,15,104386,False,False,1,0.95798,2020-04-06,0.0000,0,0,0,5,city,SP
87,Assis,3504008,2020-04-05,15,104386,False,False,1,0.95798,2020-04-05,0.0000,0,0,0,4,city,SP
88,Assis,3504008,2020-04-04,14,104386,False,False,1,0.95798,2020-04-04,0.0000,0,0,0,3,city,SP
89,Assis,3504008,2020-04-03,14,104386,False,False,1,0.95798,2020-04-03,0.0000,0,0,0,2,city,SP


In [3]:
url = "https://public.tableau.com/vizql/w/IsolamentoSocial/v/Dashboard/vudcsv/sessions/C79DEB28CCC04D6EB256BD57CDDC9644-0:0/views/3855800012607193825_2898247358194207686?summary=true"
urllib.request.urlretrieve(url, "isolamento-social-assis.csv")
isol = pd.read_csv("isolamento-social-assis.csv", delimiter=';')
isol

,Data,DIA,Média de Índice De Isolamento
0,"quinta-feira, 05/03",Quinta-feira,36%
1,"sexta-feira, 06/03",Sexta-feira,34%
2,"quinta-feira, 12/03",Quinta-feira,31%
3,"sexta-feira, 13/03",Sexta-feira,34%
4,"terça-feira, 17/03",Terça-feira,40%
...,...,...,...
105,"sexta-feira, 26/06",Sexta-feira,40%
106,"sábado, 27/06",Sábado,44%
107,"domingo, 28/06",Domingo,48%
108,"segunda-feira, 29/06",Segunda-feira,41%


## Limpando Dados

In [4]:
data['last_available_confirmed_per_100k_inhabitants'] = data['last_available_confirmed_per_100k_inhabitants'].astype(str).str.split('.', expand=True)[0]

In [5]:
columns_to_drop = ['city_ibge_code', 'estimated_population_2019', 'last_available_date', 'is_repeated',
                   'is_last', 'place_type', 'state', 'is_repeated', 'order_for_place', 'state']

In [6]:
data = data.drop(columns=columns_to_drop)

In [7]:
data['last_available_death_rate'] = data['last_available_death_rate'] * 100
data['last_available_death_rate'] = data['last_available_death_rate'].round(2)

In [8]:
data

,city,date,epidemiological_week,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_death_rate,last_available_deaths,new_confirmed,new_deaths
0,Assis,2020-07-01,27,182,174,6.04,11,0,0
1,Assis,2020-06-30,27,182,174,6.04,11,9,3
2,Assis,2020-06-29,27,173,165,4.62,8,2,0
3,Assis,2020-06-28,27,171,163,4.68,8,2,0
4,Assis,2020-06-27,26,169,161,4.73,8,2,0
...,...,...,...,...,...,...,...,...,...
86,Assis,2020-04-06,15,1,0,0.00,0,0,0
87,Assis,2020-04-05,15,1,0,0.00,0,0,0
88,Assis,2020-04-04,14,1,0,0.00,0,0,0
89,Assis,2020-04-03,14,1,0,0.00,0,0,0


In [9]:
isol['Data'] = isol['Data'].str.split(',', expand=True)[1]
isol = isol.drop(columns='DIA')
isol

,Data,Média de Índice De Isolamento
0,05/03,36%
1,06/03,34%
2,12/03,31%
3,13/03,34%
4,17/03,40%
...,...,...
105,26/06,40%
106,27/06,44%
107,28/06,48%
108,29/06,41%


## Conectando ao Google Spreadsheet e Enviando Dados

In [10]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'covid19-assis.json', scopes=scope)

gc = gspread.authorize(credentials)

In [11]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting.dataframe import format_with_dataframe, BasicFormatter

In [12]:
sheet = gc.open('Covid19 Assis')

In [13]:
set_with_dataframe(sheet.sheet1, data)

In [14]:
set_with_dataframe(sheet.get_worksheet(1), isol)